# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent. 

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/ 

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev` 
4. Install gymnasium with the needed extras

In [30]:
# %pip install -q swig
# %pip install -q gymnasium[box2d,classic_control]

## The Lunar Lander Environment 

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use 
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a `make` method to create the environment, a `reset` method, and a `step` method to execute an action.
To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [31]:
import Box2D, gymnasium as gym
print("Box2D version:", getattr(Box2D, "__version__", "unknown"))
env = gym.make("LunarLander-v3")
print("Env OK:", env)
env.close()


Box2D version: 2.3.10
Env OK: <TimeLimit<OrderEnforcing<PassiveEnvChecker<LunarLander<LunarLander-v3>>>>>


In [32]:
import gymnasium as gym

def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)
    
        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break
    
    env.close()
    return reward

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [33]:
import numpy as np

def random_agent_function(observation): 
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent_function)

Obs: [-0.00482998  1.4094263  -0.48923856 -0.0663937   0.00560353  0.11081991
  0.          0.        ] -> Action: 2
Obs: [-0.00967779  1.4086032  -0.49025673 -0.03661508  0.01101771  0.1082945
  0.          0.        ] -> Action: 2
Obs: [-1.4698791e-02  1.4086094e+00 -5.0678575e-01  2.3040459e-04
  1.5647622e-02  9.2607103e-02  0.0000000e+00  0.0000000e+00] -> Action: 1
Obs: [-0.01980362  1.4080161  -0.5172912  -0.02645038  0.02238137  0.13468717
  0.          0.        ] -> Action: 3
Obs: [-0.02482033  1.4068377  -0.50622666 -0.05245245  0.0268848   0.09007693
  0.          0.        ] -> Action: 1
Obs: [-0.029918    1.4050448  -0.51638925 -0.07981733  0.03343063  0.13092895
  0.          0.        ] -> Action: 0
Obs: [-0.03501596  1.4026525  -0.5164095  -0.10648865  0.03997406  0.13088074
  0.          0.        ] -> Action: 3
Obs: [-0.04005489  1.3996673  -0.50899494 -0.1328189   0.04502246  0.10097699
  0.          0.        ] -> Action: 2
Obs: [-0.04520493  1.3970287  -0.51970243

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [34]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [35]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:  
        return Act.MAIN.value

    return Act.NO_OP.value 

run_episode(rocket_agent_function)

Obs: [ 0.00373344  1.3983481   0.37814385 -0.55876356 -0.00431936 -0.08565535
  0.          0.        ] -> Action: 2
Obs: [ 0.00759277  1.3862478   0.38961747 -0.53780895 -0.00796098 -0.07283899
  0.          0.        ] -> Action: 2
Obs: [ 0.01158972  1.3740848   0.4027179  -0.54059494 -0.01095389 -0.05986363
  0.          0.        ] -> Action: 2
Obs: [ 0.01556101  1.3625402   0.40031272 -0.51311356 -0.01411284 -0.06318481
  0.          0.        ] -> Action: 2
Obs: [ 0.01949606  1.3516893   0.39693218 -0.4823037  -0.0175022  -0.06779335
  0.          0.        ] -> Action: 2
Obs: [ 0.02343178  1.3415009   0.3970644  -0.45285985 -0.02095832 -0.06912883
  0.          0.        ] -> Action: 2
Obs: [ 0.02747545  1.3320471   0.4074335  -0.42021286 -0.0239877  -0.06059311
  0.          0.        ] -> Action: 2
Obs: [ 0.03144789  1.3229153   0.40071553 -0.40592057 -0.02742701 -0.06879248
  0.          0.        ] -> Action: 2
Obs: [ 0.03542747  1.3146381   0.40152678 -0.36794016 -0.0309576

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [36]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)
      
        if terminated:
            break
    
    env.close()
    return reward

def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine
### Observation space
- X: vị trí ngang (trái/phải so với pad): X > 0_tàu đang lệch sang phải, X < 0_tàu đang lệch sang trái [-1.5, 1.5]
- Y: vị trí dọc (cao/thấp): Y = 0 là chạm đất
- VX: vận tốc ngang: VX > 0_trôi sang phải, VX < 0_trôi sang trái
- VY: vận tốc dọc: VY > 0_đang bay lên, VY < 0_đang rơi xuống
- ANGLE: góc nghiêng: ANGLE = 0_tàu thẳng đắng, ANGLE > 0_nghiêng sang phải, ANGLE < 0_nghiêng sang trái
- ANGULAR_VELOCITY: tốc độ xoay: >0_đang xoay chiều kim đồng hồ, <0_đang xoay ngược
- LEFT_LEG_CONTACT: chân trái chạm đất? (0 hoặc 1).
- RIGHT_LEG_CONTACT: chân phải chạm đất? (0 hoặc 1).

- tol: ngưỡng chấp nhận được

In [ ]:
# Code goes here
def run_episode_test(agent_function, seed=None):
    import gymnasium as gym
    env = gym.make("LunarLander-v3", render_mode=None)
    obs, info = env.reset(seed=seed)
    for _ in range(1000):
        action = agent_function(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        if terminated:
            break
    env.close()
    return reward

def run_episodes(agent_function, n=1000, seed0=0):
    return [run_episode_test(agent_function, seed=seed0+i) for i in range(n)]

from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

def better_agent(obs):
    X, Y, VX, VY, ANG, AV, Lc, Rc = obs
    Lc, Rc = bool(Lc), bool(Rc)

    near_ground = (Y < 0.6)
    vy_on  = -0.30 if not near_ground else -0.20

    if VY < vy_on:
        return Act.MAIN.value

    ANG_tol = 0.09 if near_ground else 0.10
    AV_hi   = 0.18 if near_ground else 0.20
    if AV > AV_hi or ANG > ANG_tol:
        return Act.RIGHT.value 
    if AV < -AV_hi or ANG < -ANG_tol:
        return Act.LEFT.value

    vx_hi = 0.35 if not near_ground else 0.16
    x_tol = 0.40 if not near_ground else 0.22
    if VX > +vx_hi:
        return Act.LEFT.value
    if VX < -vx_hi:
        return Act.RIGHT.value
    if near_ground:
        if X > +x_tol:
            return Act.LEFT.value
        if X < -x_tol:
            return Act.RIGHT.value

    if Y < 0.25:
        # giữ thăng bằng chặt
        if abs(ANG) > 0.06 or abs(AV) > 0.12:
            return Act.RIGHT.value if (ANG > 0 or AV > 0.12) else Act.LEFT.value
        # flare dọc
        if VY < -0.16:
            return Act.MAIN.value
        # nếu đã đủ chậm theo trục dọc, có thể thả
        if VY > -0.04:
            return Act.NO_OP.value
        # hãm trôi ngang nếu còn lớn
        if abs(VX) > 0.12:
            return Act.LEFT.value if VX > 0 else Act.RIGHT.value
        return Act.NO_OP.value



    return Act.NO_OP.value


rewards = run_episodes(better_agent, seed0=0)
print(rewards)
print(f"Avg={np.mean(rewards):.1f} | Success={np.sum(np.array(rewards)==100)}/{len(rewards)}")


[-100, 100, np.float64(-0.35058859598861036), -100, 100, -100, -100, 100, -100, -100, 100, np.float64(-0.04603527163086937), 100, -100, 100, 100, 100, -100, -100, -100, 100, 100, 100, 100, -100, -100, 100, -100, 100, -100, -100, -100, 100, -100, 100, -100, 100, 100, 100, 100, 100, -100, 100, 100, -100, 100, 100, 100, 100, -100, 100, 100, 100, -100, 100, -100, -100, -100, -100, 100, 100, 100, 100, -100, 100, 100, 100, -100, 100, 100, np.float64(-0.4159670510571931), -100, 100, 100, 100, -100, 100, 100, -100, -100, -100, -100, -100, np.float64(-0.010244567484703565), 100, -100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, -100, 100, 100, 100, -100, 100, -100, 100, 100, -100, -100, -100, -100, 100, -100, 100, -100, 100, 100, 100, -100, -100, -100, 100, 100, 100, 100, 100, 100, 100, -100, 100, -100, 100, -100, 100, -100, 100, -100, -100, 100, 100, 100, 100, 100, 100, -100, -100, 100, -100, -100, -100, 100, -100, 100, 100, 100, -100, 100, -100, 100, 100, -100, -100, 100, -100, 100, 100